In [12]:
# Project: Do changes in fuel prices affect political support?

# Edward Natusch - Data Science Project - University of Bristol 

# In this notebook, I will scrape presidential polling data (collected daily) for Biden from Rasmussen Reports using pandas.

# I will then clean this data.

# I will then download retail gasoline prices (collected weekly) from the FRED API.

# I will then merge this data and export it in a CSV so I can visualise it in Vega Lite.

# The retail price of gasoline (all grades all formulations) is collected every monday at 8am by the US Energy Information Administration.
# More information on their methodology can be found here: https://www.eia.gov/petroleum/gasdiesel/gas_proc-methods.php.

# The Polling Data for Rasmussen Reports Presidential Polls is collected via auomated landline telephone calls during the hours of 9am to 5pm during the week.
# More information on their methodology can be found here: https://www.rasmussenreports.com/public_content/about_us/methodology.


In [11]:
# Scraping the polling data from Rasmussen Reports using Pandas.

import pandas as pd

Biden_Approval = pd.read_html('https://www.rasmussenreports.com/public_content/politics/Biden_administration/Biden_approval_index_history')

# Loading and cleaning the data 

DF_Biden_Approval = Biden_Approval[0]

DF_Biden_Approval["Date"] = pd.to_datetime(DF_Biden_Approval["Date"])

DF_Biden_Approval['Total Approve'] = DF_Biden_Approval['Total Approve'].str.replace('%',' ')
DF_Biden_Approval['Total Approve'] = pd.to_numeric(DF_Biden_Approval['Total Approve'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Total Approve"], inplace=True)

DF_Biden_Approval['Total Disapprove'] = DF_Biden_Approval['Total Disapprove'].str.replace('%',' ')
DF_Biden_Approval['Total Disapprove'] = pd.to_numeric(DF_Biden_Approval['Total Disapprove'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Total Disapprove"], inplace=True)

DF_Biden_Approval['Strongly Approve'] = DF_Biden_Approval['Strongly Approve'].str.replace('%',' ')
DF_Biden_Approval['Strongly Approve'] = pd.to_numeric(DF_Biden_Approval['Strongly Approve'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Strongly Approve"], inplace=True)


DF_Biden_Approval['Strongly Disapprove'] = DF_Biden_Approval['Strongly Disapprove'].str.replace('%',' ')
DF_Biden_Approval['Strongly Disapprove'] = pd.to_numeric(DF_Biden_Approval['Strongly Disapprove'], errors ='coerce')
DF_Biden_Approval.dropna(subset = ["Strongly Disapprove"], inplace=True)

In [10]:
# Importing the required packages to download the data from the FRED API.

import requests

import os 

import json

from google.colab import files

In [9]:
# Downloading the Json file from th Fred API and loading it into a pandas data frame 

url = 'https://api.stlouisfed.org/fred/series/observations?series_id=GASALLW&api_key=e725511ce5615fe46441618aca9c09bf&file_type=json'

data = requests.get(url)

Retail_Gasoline_Consumer_Price = data.json()

Observations = Retail_Gasoline_Consumer_Price['observations']

Retail_Gasoline_Consumer_Price_Array=[]

for x in Observations:
  temp_variable={'date':x['date'],'value':x['value']}
  Retail_Gasoline_Consumer_Price_Array.append(temp_variable)

DF_Retail_Gasoline_Price  = pd.DataFrame(Retail_Gasoline_Consumer_Price_Array)

# Adding New Column Names 

DF_Retail_Gasoline_Price.columns = ['Date','Retail Price of Gasoline (Dollars per Gallon)']

# Formatting Dates

DF_Retail_Gasoline_Price['Date'] = pd.to_datetime(DF_Retail_Gasoline_Price['Date'])

# Formatting Price Index

DF_Retail_Gasoline_Price['Retail Price of Gasoline (Dollars per Gallon)'] = pd.to_numeric(DF_Retail_Gasoline_Price['Retail Price of Gasoline (Dollars per Gallon)'], errors ='coerce')


In [8]:
# Merging the data from the two data sources to create a single data set 

DF_Biden_Approval_Weekly_With_Gasoline_Prices = DF_Biden_Approval.merge(DF_Retail_Gasoline_Price, left_on='Date', right_on='Date')

# Creating a month field

DF_Biden_Approval_Weekly_With_Gasoline_Prices['Month'] = pd.DatetimeIndex(DF_Biden_Approval_Weekly_With_Gasoline_Prices['Date']).month

# Exporting the data to csv

DF_Biden_Approval_Weekly_With_Gasoline_Prices.to_csv("Project_President_Biden_Polling_Weekly_Retail_Price_of_Gasoline_Without_Lag.csv")
